In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from datetime import datetime
from scipy import stats

In [3]:
# # read data
# # trial with one city
# earth_weather_data = db_connect.earth_weather_data()
# mars_weather_data = db_connect.mars_weather_data()
# solar_flares_data = db_connect.solar_flares_data()

In [4]:
# earth_weather_data.info()

In [5]:
# earth_weather_data.head()

In [6]:
# earth_weather_data.date.min(), earth_weather_data.date.max()

In [7]:
# earth_weather_data['date'] = pd.to_datetime(earth_weather_data['date'])

# earth_weather_data['date'] = earth_weather_data['date'].dt.normalize()

# # Group by the date and calculate the mean for each column of interest
# daily_avg = earth_weather_data.groupby('date').agg({
#     'latitude': 'first',
#     'longitude': 'first',
#     'temperature_2m': 'mean',
#     'relative_humidity_2m': 'mean',
#     'rain': 'mean',
#     'direct_radiation_instant': 'mean',
#     'location': 'first'
# }).reset_index()

# daily_avg.head()

In [8]:
# # monthly average
# earth_weather_data_monthagg = earth_weather_data.copy(deep=True)
# earth_weather_data_monthagg.set_index('date', inplace=True)
# aggregations = {
#     'temperature_2m': 'mean',
#     'relative_humidity_2m': 'mean',
#     'rain': 'mean',
#     'direct_radiation_instant': 'mean',
# }
# monthly_avg = earth_weather_data_monthagg.groupby([pd.Grouper(freq='M'), 'location']).agg(aggregations).reset_index()
# monthly_avg.to_parquet('data/earth_monthlyavg.parquet.gzip',compression='gzip')
# monthly_avg.info()

In [9]:
# # annual average
# earth_weather_data_yearagg = earth_weather_data.copy(deep=True)
# earth_weather_data_yearagg.set_index('date', inplace=True)
# year_avg = earth_weather_data_monthagg.groupby([pd.Grouper(freq='A'), 'location']).agg(aggregations).reset_index()
# year_avg.to_parquet('data/earth_yearavg.parquet.gzip',compression='gzip')
# year_avg.info()

In [10]:
# mars_weather_data.to_parquet('data/mars.parquet.gzip',compression='gzip')
# solar_flares_data.to_parquet('data/flare.parquet.gzip',compression='gzip')

In [11]:
# daily_avg.to_parquet('data/df.parquet.gzip',compression='gzip')

In [12]:
mars_weather_data = pd.read_parquet('data/mars.parquet.gzip')
solar_flares_data = pd.read_parquet('data/flare.parquet.gzip')
daily_avg = pd.read_parquet('data/df.parquet.gzip')

In [13]:
mars_weather_data.head()

,primary_key,id,terrestrial_date,sol,ls,month,min_temp,max_temp,pressure,atmo_opacity
0,19589,1895,2018-02-27,1977,135,Month 5,-77.0,-10.0,727.0,Sunny
1,19590,1893,2018-02-26,1976,135,Month 5,-77.0,-10.0,728.0,Sunny
2,19591,1894,2018-02-25,1975,134,Month 5,-76.0,-16.0,729.0,Sunny
3,19592,1892,2018-02-24,1974,134,Month 5,-77.0,-13.0,729.0,Sunny
4,19593,1889,2018-02-23,1973,133,Month 5,-78.0,-18.0,730.0,Sunny


In [14]:
mars_weather_data.terrestrial_date.min(), mars_weather_data.terrestrial_date.max()

(Timestamp('2012-08-16 00:00:00'), Timestamp('2022-11-25 00:00:00'))

In [15]:
solar_flares_data.head()

,id,flrid,instruments,begintime,peaktime,endtime,classtype,sourcelocation,activeregionnum,note,linkedevents,submissiontime,link,intensity
0,1144,2023-10-01T01:10:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2023-10-01 01:10:00,2023-10-01 01:32:00,2023-10-01 01:43:00,M2.5,N12E58,13452.0,None,None,2023-10-01T14:10Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,Medium
1,1145,2023-10-01T01:44:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2023-10-01 01:44:00,2023-10-01 01:46:00,2023-10-01 01:49:00,M1.6,N15E59,13451.0,None,None,2023-10-01T01:59Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,Medium
2,1146,2023-10-02T12:35:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2023-10-02 12:35:00,2023-10-02 12:46:00,2023-10-02 12:58:00,M1.9,N19E68,13455.0,None,None,2023-10-04T15:53Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,Medium
3,1147,2023-10-02T15:53:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2023-10-02 15:53:00,2023-10-02 15:58:00,2023-10-02 16:02:00,C2.4,S17E02,13450.0,None,[{'activityID': '2023-10-02T18:24:00-CME-001'}],2023-10-03T17:19Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,Small
4,1148,2023-10-07T17:57:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2023-10-07 17:57:00,2023-10-07 18:06:00,2023-10-07 18:10:00,M1.7,S12E28,13457.0,None,None,2023-10-07T18:24Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,Medium


In [16]:
# Select Relevant Features:
mars_weather_data = mars_weather_data[["terrestrial_date", "min_temp", "max_temp", "pressure", "atmo_opacity"]]
earth_weather_data_daily = daily_avg[["date", "temperature_2m", "relative_humidity_2m", "rain", "direct_radiation_instant"]]
solar_flares_data = solar_flares_data[["peaktime", "classtype", "intensity"]]

# Rename Columns for Consistency:
mars_weather_data.rename(columns={"terrestrial_date": "date"}, inplace=True)
solar_flares_data.rename(columns={"peaktime": "date"}, inplace=True)

In [17]:
mars_weather_data['date'] = pd.to_datetime(mars_weather_data['date']).dt.date
earth_weather_data_daily['date'] = pd.to_datetime(earth_weather_data_daily['date']).dt.date
solar_flares_data['date'] = pd.to_datetime(solar_flares_data['date'], errors = 'coerce').dt.date

C:\Users\ansam\AppData\Local\Temp\ipykernel_20616\3253603673.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  earth_weather_data_daily['date'] = pd.to_datetime(earth_weather_data_daily['date']).dt.date


In [18]:
solar_flares_data.columns

Index(['date', 'classtype', 'intensity'], dtype='object')

In [19]:
# Merge DataFrames:
merged_df = pd.merge(mars_weather_data, earth_weather_data_daily, on="date", how="left")
merged_df = pd.merge(merged_df, solar_flares_data, on="date", how="left")

In [20]:
merged_df.classtype.value_counts()

classtype
M1.0    108
M1.1     88
M1.2     73
M1.3     63
M1.4     53
       ... 
C2.0      1
B8.5      1
B7.5      1
B7.2      1
B6.1      1
Name: count, Length: 199, dtype: int64

In [21]:
merged_df['date'] = pd.to_datetime(merged_df['date']).dt.date
merged_df = merged_df.sort_values('date', ascending=False)
merged_df['solar_flare'] = merged_df['classtype'].notna().astype(int)
merged_df.head()


,date,min_temp,max_temp,pressure,atmo_opacity,temperature_2m,relative_humidity_2m,rain,direct_radiation_instant,classtype,intensity,solar_flare
2079,2022-11-25,-72.0,-44.0,817.0,Sunny,1.209279,62.662373,0.017949,107.730278,NaN,NaN,0
2080,2022-11-24,-72.0,-9.0,800.0,Sunny,1.779150,61.529714,0.096955,97.351560,NaN,NaN,0
2081,2022-11-23,-73.0,-8.0,797.0,Sunny,3.125305,61.220017,0.080929,96.340043,NaN,NaN,0
2082,2022-11-22,-72.0,-10.0,798.0,Sunny,3.216731,59.682723,0.042308,111.963284,C7.0,Small,1
2083,2022-11-22,-72.0,-10.0,798.0,Sunny,3.216731,59.682723,0.042308,111.963284,C4.8,Small,1


In [22]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5450 entries, 2079 to 2078
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      5450 non-null   object 
 1   min_temp                  5449 non-null   float64
 2   max_temp                  5449 non-null   float64
 3   pressure                  5450 non-null   float64
 4   atmo_opacity              5450 non-null   object 
 5   temperature_2m            5450 non-null   float64
 6   relative_humidity_2m      5450 non-null   float64
 7   rain                      5450 non-null   float64
 8   direct_radiation_instant  5450 non-null   float64
 9   classtype                 1259 non-null   object 
 10  intensity                 1259 non-null   object 
 11  solar_flare               5450 non-null   int32  
dtypes: float64(7), int32(1), object(4)
memory usage: 532.2+ KB


In [23]:
merged_df['intensity'] = merged_df['intensity'].fillna('None')
merged_df['classtype'] = merged_df['classtype'].fillna('None')

In [24]:
merged_df.tail(20)

,date,min_temp,max_temp,pressure,atmo_opacity,temperature_2m,relative_humidity_2m,rain,direct_radiation_instant,classtype,intensity,solar_flare
2059,2012-09-08,-73.0,-2.0,748.0,Sunny,14.842394,58.813843,0.003205,161.299413,None,None,0
2060,2012-09-06,-74.0,-23.0,745.0,Sunny,14.764750,61.777769,0.044872,143.207505,None,None,0
2061,2012-09-05,-74.0,-3.0,747.0,Sunny,15.907619,60.702755,0.006571,134.559424,None,None,0
2062,2012-09-04,-75.0,-2.0,747.0,Sunny,15.365712,59.648517,0.036859,136.920584,None,None,0
2063,2012-09-03,-75.0,-15.0,745.0,Sunny,14.526369,59.409648,0.007692,151.811408,None,None,0
2064,2012-09-02,-75.0,-15.0,743.0,Sunny,14.071080,62.557399,0.025000,136.166256,None,None,0
2065,2012-09-01,-76.0,-14.0,745.0,Sunny,14.708500,59.976835,0.074359,139.646827,None,None,0
2066,2012-08-31,-75.0,-11.0,743.0,Sunny,14.527410,59.542672,0.076763,143.821035,C8.4,Small,1
2067,2012-08-30,-75.0,-7.0,742.0,Sunny,15.331817,57.950106,0.073397,142.182245,None,None,0
2068,2012-08-29,-75.0,-16.0,741.0,Sunny,15.426128,61.081991,0.106410,151.402620,None,None,0


In [25]:
# # Scale Data for Forecasting
# scaler = MinMaxScaler()
# merged_df[["min_temp", "max_temp", "pressure", "temperature_2m"]] = scaler.fit_transform(merged_df[["min_temp", "max_temp", "pressure", "temperature_2m"]])

In [26]:
# Time Series Plots:
merged_df['temperature_2m_smooth'] = merged_df['temperature_2m'].rolling(window=7).mean()

fig = px.scatter(merged_df, x="date", y="temperature_2m", 
                  title="Time Series of Earth Temperature Data")
fig.update_layout(
    # paper_bgcolor='rgba(0,0,0,0)',
    #               plot_bgcolor='rgba(0,0,0,0)',
    #               font=dict(color='white'),
                  height=1000, 
                  width=2000)
fig.show()

In [27]:
fig = px.scatter(merged_df, x="date", y="max_temp", 
                  title="Time Series of Mar's Max Temperature Data")
fig.update_layout(
    # paper_bgcolor='rgba(0,0,0,0)',
    #               plot_bgcolor='rgba(0,0,0,0)',
    #               font=dict(color='white'),
                  height=1000, 
                  width=2000)
fig.show()

In [28]:
# Create traces
trace_earth = go.Scatter(
    x=merged_df['date'],
    y=merged_df['temperature_2m'],
    mode='lines',
    name='Earth',
    line=dict(color='blue')
)

trace_mars = go.Scatter(
    x=merged_df['date'],
    y=merged_df['max_temp'],
    mode='lines',
    name='Mars',
    line=dict(color='red', dash='dot')  # Different style for Mars for clarity
)

fig = go.Figure(data=[trace_earth, trace_mars])
fig.update_layout(
    title='Comparative Temperature Trends on Earth and Mars (2014-2022)',
    xaxis_title='Year',
    yaxis_title='Temperature (°C)',
    # template='plotly_dark'  # Using dark theme for better visibility
)

fig.show()

In [29]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5450 entries, 2079 to 2078
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      5450 non-null   object 
 1   min_temp                  5449 non-null   float64
 2   max_temp                  5449 non-null   float64
 3   pressure                  5450 non-null   float64
 4   atmo_opacity              5450 non-null   object 
 5   temperature_2m            5450 non-null   float64
 6   relative_humidity_2m      5450 non-null   float64
 7   rain                      5450 non-null   float64
 8   direct_radiation_instant  5450 non-null   float64
 9   classtype                 5450 non-null   object 
 10  intensity                 5450 non-null   object 
 11  solar_flare               5450 non-null   int32  
 12  temperature_2m_smooth     5444 non-null   float64
dtypes: float64(8), int32(1), object(4)
memory usage: 574.8+ KB


In [30]:
merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month
pivot_earth = merged_df.pivot_table(values='temperature_2m', index='year', columns='month', aggfunc='mean')
pivot_mars = merged_df.pivot_table(values='max_temp', index='year', columns='month', aggfunc='mean')

In [31]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Heatmap of Earth Temperature Changes by Month and Year', 'Heatmap of Mars Temperature Changes by Month and Year')
)

fig.add_trace(
    go.Heatmap(
        z=pivot_earth.values,
        x=pivot_earth.columns,
        y=pivot_earth.index,
        colorscale='RdBu_r',
        colorbar=dict(len=0.5, x=0.46, title='Temp °C'),  # half length colorbar positioned between subplots
        showscale=True
    ),
    row=1, col=1
)

fig.add_trace(
    go.Heatmap(
        z=pivot_mars.values,
        x=pivot_mars.columns,
        y=pivot_mars.index,
        colorscale='RdBu_r',
        colorbar=dict(len=0.5, x=1.0, title='Temp °C'),  # half length colorbar on the right side
        showscale=True
    ),
    row=1, col=2
)

fig.update_layout(
    title_text='Comparative Heatmaps of Temperature Changes by Month and Year',
    height=600, width=1200
)

fig.show()

In [32]:
import pandas as pd
import numpy as np
from scipy import stats

merged_df2 = merged_df.copy(deep=True).dropna()
merged_df2['date'] = pd.to_datetime(merged_df2['date'])
# Set 'date' as the index for easier slicing by date/time
merged_df2.set_index('date', inplace=True)

# Calculate mean, median, and mode for Earth
mean_earth = merged_df2['temperature_2m'].mean()
median_earth = merged_df2['temperature_2m'].median()
mode_earth = merged_df2['temperature_2m'].mode()[0]  # mode() returns a Series

mean_mars = merged_df2['max_temp'].mean()
median_mars = merged_df2['max_temp'].median()
mode_mars = merged_df2['max_temp'].mode()[0]

print(f"Earth - Mean: {mean_earth}, Median: {median_earth}, Mode: {mode_earth}")
print(f"Mars - Mean: {mean_mars}, Median: {median_mars}, Mode: {mode_mars}")

# print(merged_df2.isnull().sum())  
# print(merged_df2.index)

# Seasonality analysis
earth_seasonal = merged_df2['temperature_2m'].resample('Q').mean()  # Quarterly means
mars_seasonal = merged_df2['max_temp'].resample('Q').mean()

print("Earth Seasonal Temperature Means:")
print(earth_seasonal)
print("Mars Seasonal Temperature Means:")
print(mars_seasonal)

# Volatility: standard deviation as a measure of volatility
volatility_earth = merged_df2['temperature_2m'].std()
volatility_mars = merged_df2['max_temp'].std()

print(f"Volatility in Earth's temperature: {volatility_earth}")
print(f"Volatility in Mars' temperature: {volatility_mars}")

Earth - Mean: 8.548246380613817, Median: 8.669535304872596, Mode: 12.765711608799359
Mars - Mean: -12.739665625574132, Median: -12.0, Mode: -4.0
Earth Seasonal Temperature Means:
date
2012-09-30    14.488740
2012-12-31     5.147633
2013-03-31     0.320798
2013-06-30    12.373227
2013-09-30    15.431047
2013-12-31     4.995584
2014-03-31     2.123650
2014-06-30    12.874406
2014-09-30    16.036000
2014-12-31     5.059725
2015-03-31     2.358555
2015-06-30    11.357306
2015-09-30    15.383260
2015-12-31     4.356102
2016-03-31     2.229509
2016-06-30    12.716244
2016-09-30    15.541275
2016-12-31     4.442822
2017-03-31     1.319523
2017-06-30    11.601091
2017-09-30    15.203221
2017-12-31     4.126568
2018-03-31     1.740575
2018-06-30    12.180630
2018-09-30    16.097275
2018-12-31     3.061848
2019-03-31     1.098615
2019-06-30    11.791300
2019-09-30    15.589329
2019-12-31     4.469367
2020-03-31     1.294972
2020-06-30    11.617446
2020-09-30    15.666769
2020-12-31     4.605984


In [33]:
# Time Series Plots:
merged_df['temperature_2m_smooth'] = merged_df['temperature_2m'].rolling(window=7).mean()

fig = make_subplots(rows=4, cols=1, shared_xaxes=True,
                    subplot_titles=("Mars Min Temp", "Mars Max Temp", "Mars Pressure", "Earth Temperature"))

# Plot each variable on a separate subplot
fig.add_trace(go.Scatter(x=merged_df['date'], y=merged_df['min_temp'], name='Mars Min Temp'), row=1, col=1)
fig.add_trace(go.Scatter(x=merged_df['date'], y=merged_df['max_temp'], name='Mars Max Temp'), row=2, col=1)
fig.add_trace(go.Scatter(x=merged_df['date'], y=merged_df['pressure'], name='Mars Pressure'), row=3, col=1)
fig.add_trace(go.Scatter(x=merged_df['date'], y=merged_df['temperature_2m_smooth'], name='Earth Temperature Smoothed'), row=4, col=1)

fig.update_xaxes(title_text="Date", row=4, col=1, rangeslider_visible=True, rangeselector=dict(
    buttons=list([
        dict(count=1, label="1y", step="year", stepmode="backward"),
        dict(count=2, label="2y", step="year", stepmode="backward"),
        dict(count=5, label="5y", step="year", stepmode="backward"),
        dict(step="all")
    ])
))

# Update x-axis and y-axis labels
fig.update_xaxes(title_text="Date", row=4, col=1)  # Only the last subplot needs the x-axis label
fig.update_yaxes(title_text="Degrees C", row=1, col=1)
fig.update_yaxes(title_text="Degrees C", row=2, col=1)
fig.update_yaxes(title_text="Pressure (Pa)", row=3, col=1)
fig.update_yaxes(title_text="Degrees C", row=4, col=1)

fig.update_layout(
    # paper_bgcolor='rgba(0,0,0,0)',
    #               plot_bgcolor='rgba(0,0,0,0)',
                #   font=dict(color='white'),
                  width=1500, 
                  height=1000, 
                  title_text="Time Series of Mars and Earth Weather Data")
fig.show()

In [34]:
merged_df.columns

Index(['date', 'min_temp', 'max_temp', 'pressure', 'atmo_opacity',
       'temperature_2m', 'relative_humidity_2m', 'rain',
       'direct_radiation_instant', 'classtype', 'intensity', 'solar_flare',
       'temperature_2m_smooth', 'year', 'month'],
      dtype='object')

In [35]:
merged_df.head()

,date,min_temp,max_temp,pressure,atmo_opacity,temperature_2m,relative_humidity_2m,rain,direct_radiation_instant,classtype,intensity,solar_flare,temperature_2m_smooth,year,month
2079,2022-11-25,-72.0,-44.0,817.0,Sunny,1.209279,62.662373,0.017949,107.730278,None,None,0,NaN,2022,11
2080,2022-11-24,-72.0,-9.0,800.0,Sunny,1.779150,61.529714,0.096955,97.351560,None,None,0,NaN,2022,11
2081,2022-11-23,-73.0,-8.0,797.0,Sunny,3.125305,61.220017,0.080929,96.340043,None,None,0,NaN,2022,11
2082,2022-11-22,-72.0,-10.0,798.0,Sunny,3.216731,59.682723,0.042308,111.963284,C7.0,Small,1,NaN,2022,11
2083,2022-11-22,-72.0,-10.0,798.0,Sunny,3.216731,59.682723,0.042308,111.963284,C4.8,Small,1,NaN,2022,11


In [36]:
df_mars = merged_df[['date', 'max_temp']]
df_earth = merged_df[['date', 'temperature_2m']]

df_mars['date'] = pd.to_datetime(df_mars['date'])
df_earth['date'] = pd.to_datetime(df_earth['date'])

df_mars.set_index('date', inplace=True)
df_earth.set_index('date', inplace=True)

# Resample data by monthly mean
mars_monthly = df_mars.resample('M').mean()
earth_monthly = df_earth.resample('M').mean()

# Pearson correlation
correlation_temp = mars_monthly['max_temp'].corr(earth_monthly['temperature_2m'])
print(f"Correlation between Mars max temp and Earth temp: {correlation_temp}")

Correlation between Mars max temp and Earth temp: 0.008070866864465013


C:\Users\ansam\AppData\Local\Temp\ipykernel_20616\411139898.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ansam\AppData\Local\Temp\ipykernel_20616\411139898.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [37]:
mars_data = mars_weather_data[["date", "min_temp", "max_temp", "pressure"]]
earth_data = daily_avg[["date", "temperature_2m", "relative_humidity_2m", "rain", "direct_radiation_instant"]]

mars_data['date'] = pd.to_datetime(mars_data['date'])
earth_data['date'] = pd.to_datetime(earth_data['date'])

# Assuming you have 'mars_data' and 'earth_data' DataFrames
correlation_matrix = pd.concat([mars_data, earth_data], axis=1).corr()  # Combine and calculate correlation
fig = px.imshow(correlation_matrix, color_continuous_scale='RdBu_r') # Create heatmap
fig.update_layout(title="Correlation Heatmap: Mars vs. Earth Weather", width=800, height=600)
fig.show()

C:\Users\ansam\AppData\Local\Temp\ipykernel_20616\2501049734.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ansam\AppData\Local\Temp\ipykernel_20616\2501049734.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

merged_df = merged_df.dropna()
X = merged_df[['min_temp', 'pressure']] 
y = merged_df['temperature_2m'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)


fig = go.Figure()
fig.add_trace(go.Scatter(x=merged_df['date'], y=y_test, name='Actual Earth Temperature', mode='lines+markers'))
fig.add_trace(go.Scatter(x=merged_df['date'], y=predictions, name='Predicted Earth Temperature', mode='lines+markers'))

# Calculate Error
mse = mean_squared_error(y_test, predictions)

fig.update_layout(title=f"Linear Regression Model:  MSE = {mse:.2f}",
                  xaxis=dict(title="Date"),
                  yaxis=dict(title="Temperature (Celsius)"))
fig.show()

In [39]:
from sklearn.linear_model import LinearRegression

df_mars = merged_df[['date', 'max_temp', 'pressure']]
df_earth = merged_df[['date', 'temperature_2m']]

df_mars['date'] = pd.to_datetime(df_mars['date'])
df_earth['date'] = pd.to_datetime(df_earth['date'])

df_mars.set_index('date', inplace=True)
df_earth.set_index('date', inplace=True)

# Resample data by monthly mean
mars_monthly = df_mars.resample('M').mean()
earth_monthly = df_earth.resample('M').mean()

X = mars_monthly[['max_temp', 'pressure']]  # Predictor variables from Mars
y = earth_monthly['temperature_2m']  # Target variable from Earth

# Regression model
model = LinearRegression()
model.fit(X, y)
print("Coefficient:", model.coef_, "Intercept:", model.intercept_)


Coefficient: [0.00829759 0.00844555] Intercept: 1.609243268436094


C:\Users\ansam\AppData\Local\Temp\ipykernel_20616\1283478925.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ansam\AppData\Local\Temp\ipykernel_20616\1283478925.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Effect of solar flares on the two planets

In [94]:
df_mars_weather = pd.read_parquet('data/mars.parquet.gzip')
df_earth_weather = pd.read_parquet('data/df.parquet.gzip')
df_solar_flares = pd.read_parquet('data/flare.parquet.gzip')

intensity_mapping = {
    'Small': 1,
    'Medium': 2,
    'Most intense': 3
}

df_solar_flares['intensity_numeric'] = df_solar_flares['intensity'].map(intensity_mapping)

df_mars_weather['terrestrial_date'] = pd.to_datetime(df_mars_weather['terrestrial_date']).dt.date
df_earth_weather['date'] = pd.to_datetime(df_earth_weather['date']).dt.date
df_solar_flares['peaktime'] = pd.to_datetime(df_solar_flares['peaktime'], errors = 'coerce').dt.date

df_mars_weather['terrestrial_date'] = pd.to_datetime(df_mars_weather['terrestrial_date'])
df_earth_weather['date'] = pd.to_datetime(df_earth_weather['date'])
df_solar_flares['peaktime'] = pd.to_datetime(df_solar_flares['peaktime'], errors = 'coerce')

df_mars_weather.set_index('terrestrial_date', inplace=True)
df_earth_weather.set_index('date', inplace=True)
df_solar_flares.set_index('peaktime', inplace=True)

df_mars_weather.drop(columns=['month', 'atmo_opacity', 'primary_key', 'id'], inplace=True)
df_earth_weather.drop(columns=['location', 'latitude', 'longitude'], inplace=True)

df_mars_daily = df_mars_weather.resample('D').mean()
df_earth_daily = df_earth_weather.resample('D').mean()

In [98]:
# Correlation analysis function
def analyze_impact(weather_data, solar_flare_data, planet_name):
    # Merge datasets on the index (date)
    combined_data = pd.merge(weather_data, solar_flare_data, left_index=True, right_index=True, how='inner')
    # Calculate correlations
    correlation_results = combined_data.corr()

    print("Correlation Matrix:")
    print(correlation_results)

    fig = px.imshow(correlation_results, text_auto=True,
                    labels=dict(color='Correlation'),
                    title=f"Correlation Heatmap between Solar Flares and Weather Data on {planet_name}", width=800, height=600)
    fig.show()

In [99]:
# Analyzing impact on Mars
analyze_impact(df_mars_daily, df_solar_flares[['intensity_numeric']], 'Mars')

Correlation Matrix:
                        sol        ls  min_temp  max_temp  pressure  \
sol                1.000000  0.241405  0.528751  0.198107 -0.376247   
ls                 0.241405  1.000000  0.685948  0.687026 -0.111158   
min_temp           0.528751  0.685948  1.000000  0.805662 -0.193587   
max_temp           0.198107  0.687026  0.805662  1.000000 -0.203431   
pressure          -0.376247 -0.111158 -0.193587 -0.203431  1.000000   
intensity_numeric -0.315657 -0.025541 -0.054720  0.071709  0.170609   

                   intensity_numeric  
sol                        -0.315657  
ls                         -0.025541  
min_temp                   -0.054720  
max_temp                    0.071709  
pressure                    0.170609  
intensity_numeric           1.000000  


In [100]:
# Analyzing impact on Earth
analyze_impact(df_earth_daily, df_solar_flares[['intensity_numeric']], 'Earth')

Correlation Matrix:
                          temperature_2m  relative_humidity_2m      rain  \
temperature_2m                  1.000000             -0.641420  0.047739   
relative_humidity_2m           -0.641420              1.000000  0.136511   
rain                            0.047739              0.136511  1.000000   
direct_radiation_instant        0.731656             -0.720908 -0.025506   
intensity_numeric               0.006977              0.018436 -0.056475   

                          direct_radiation_instant  intensity_numeric  
temperature_2m                            0.731656           0.006977  
relative_humidity_2m                     -0.720908           0.018436  
rain                                     -0.025506          -0.056475  
direct_radiation_instant                  1.000000           0.022307  
intensity_numeric                         0.022307           1.000000  
